In [1]:
import keras
from keras.models import Sequential
from keras import layers
from keras.layers import Dense,InputLayer, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import pandas as pd
import datetime

import numpy as np
import tensorflow as tf
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
import keras_tuner
from keras.metrics import categorical_accuracy
from tensorflow.keras import regularizers
from sklearn import preprocessing
import multiprocessing


2024-01-31 21:12:06.891906: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 21:12:06.894509: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-31 21:12:06.940094: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 21:12:06.940131: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 21:12:06.940155: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
import tensorflow as tf
from tensorflow.python.eager import context

_ = tf.Variable([48])

context._context = None
context._create_context()

tf.config.threading.set_inter_op_parallelism_threads(48)

2024-01-05 05:32:00.331464: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# Global Model

In [2]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self,hp):
        model = Sequential()
        model.add(InputLayer(input_shape=(240, ))) #240 timestamps with 1 feature
        hp_units = hp.Choice('units', values=[32,64,128])
        model.add(Dense(units=hp_units, name='dense_0'))
        model.add(Dropout(hp.Choice('dropout_rate',values =[0.5])))
        model.add(Dense(1, activation='sigmoid'))
        hp_lr = hp.Choice('learning_rate', values=[1e-6, 1e-7, 1e-8])
        hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam',"adamax"])
        if hp_optimizer == 'sgd':
            optimizer = keras.optimizers.SGD(learning_rate=hp_lr)
        elif hp_optimizer == 'rmsprop':
            optimizer = keras.optimizers.RMSprop(learning_rate=hp_lr)
        elif hp_optimizer == 'adam':
            optimizer = keras.optimizers.Adam(learning_rate=hp_lr)
        elif hp_optimizer == 'adamax':
            optimizer = keras.optimizers.Adamax(learning_rate=hp_lr)
        else:
            raise ValueError("Invalid optimizer choice")

        model.compile(
            optimizer = optimizer
            ,loss=keras.losses.BinaryCrossentropy()
            , metrics=['accuracy'])
        return model


    def fit(self, hp, model, *args, **kwargs):
            return model.fit(
                *args,
                batch_size=hp.Choice("batch_size", [16,32, 64,128,256]),
                **kwargs,
            )


In [3]:
tuner = keras_tuner.BayesianOptimization(MyHyperModel(),
                objective='val_accuracy',  overwrite=True,
                max_trials=30, directory='fnn', seed=11)
print(tuner.search_space_siumm)


KeyboardInterrupt



In [ ]:
timesteps = 240
num_input =1
num_classes=1
label = list(range(timesteps)) + ['target'] + ['ticker'] + ['target_date'] + ['sector']

training_data = []
training_label = []
testing_data =[]
testing_label =[]

accuracy_results = []

for i in range(5):
    # read the data
    data_path = '/DAX/data/'
    train = pd.read_csv(data_path+'Set_' + str(i) + '_Train.csv', index_col=0).dropna()
    test = pd.read_csv(data_path+'Set_' + str(i) + '_Test.csv', index_col=0).dropna()

    train.columns = label
    test.columns = label

    train_label = train.iloc[:, timesteps]
    train_data = train.iloc[:, :timesteps]
    test_label = test.iloc[:,timesteps]
    test_data = test.iloc[:, :timesteps]
    
    
     # reshape input
    #  data: (samples, timesteps, features)
    x_train = np.array(train_data).reshape((len(train_data), timesteps, num_input), order = 'F')
    x_test = np.array(test_data).reshape((len(test_data), timesteps, num_input), order = 'F')
    # label: (samples, target)
    y_train = np.array(train_label).reshape((len(train_label), num_classes))
    y_test = np.array(test_label).reshape((len(test_label), num_classes))
        
    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    
    print("-------------------------------------------------------------------------------------------------------")
    print("Training the model for Training Set " + str(i) + " from " +
    datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'))
    print("-------------------------------------------------------------------------------------------------------")
        
        
    if i==0:    
        tuner = keras_tuner.BayesianOptimization(MyHyperModel(),
                objective='val_accuracy',  #overwrite=True,
                max_trials=30, directory='fnn', seed=11)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights=False)
        tuner.search(x_train,y_train, epochs =1000,validation_split=0.2,callbacks=[early_stop])

        # save the best model
        hypermodel = MyHyperModel()
        best_hp = tuner.get_best_hyperparameters()[0]
        print(tuner.get_best_hyperparameters()[0].get_config()["values"])
        best_model = hypermodel.build(best_hp)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights=False)
        result = best_model.fit(x_train,y_train, epochs=1000, validation_split =0.2, verbose =1, callbacks=[early_stop])


    else:
        load_path="n_weight.h5"
        print('Model restore from '+load_path)
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=load_path,
                                                     save_weights_only=True,
                                                     verbose=1)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                        patience = 10)
        result = best_model.fit(
                x_train, 
                y_train, 
                epochs = 1000, 
                validation_split=0.2,
                verbose =1,
                callbacks=[cp_callback, early_stop]        
            )         
      
      
        
    save_path = "/DAX/4_FNN/fnn_model_weight/fnn_weight.h5"
    best_model.save_weights(save_path)
    print("Model saved to " + save_path)
    print("Training end: " + datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'))
        
    ##make prediction
    pred_ff_test = best_model.predict(x_test)
    #pred = pred_ff_test.tolist()
    pred = pred_ff_test.reshape((1, len(pred_ff_test))).tolist()[0]
    output_data = pd.DataFrame({'y_prob': pred, 'y_true': test['target'], 'Ticker': test['ticker'],
                                    'Date': test['target_date'], 'Sector': test['sector'], })
    accuracy = accuracy_score(np.round(output_data['y_prob']), output_data['y_true'])
    print('Overall Accuracy for test set:'+ str(accuracy))
    output_data.to_csv('prediction_period_'+str(i)+'.csv')
    print('Prediction for period ' + str(i) + ' successfully saved.')

# One model for each Sector

In [9]:
sector = pd.read_csv('/DAX/8_performance/stock_analysis.csv')
sector_list = sector["Sector"].unique().tolist()
#sector_list.remove('Academic & Educational Services')
print(sector_list)

['Basic Materials', 'Consumer Cyclicals', 'Financials', 'Healthcare', 'Consumer Non-Cyclicals', 'Industrials', 'Technology', 'Real Estate', 'Utilities']


In [16]:
combined_data_sector = pd.DataFrame()

# Loop through the files and append their conten
for sector in sector_list:
    for i in range(23):  
        path = 'fnn_'+sector+'_prediction_period_'+str(i)+'.csv'
        data = pd.read_csv(path)  
        # data = pd.read_csv('fnn_prediction_period_' + str(i) + '.csv')
        combined_data_sector = pd.concat([combined_data_sector,data], ignore_index=True)

# Save the combined data to a single CSV file
combined_data_sector.to_csv('fnn_sector_prediction.csv', index=False)

In [16]:
accuracy_results

{'Healthcare': {0: 0.5088546578644032,
  1: 0.50144030726555,
  2: 0.5090358503467813,
  3: 0.5078192458984937,
  4: 0.504073680481757,
  5: 0.5087660711304057,
  6: 0.4978799185888738,
  7: 0.49211006264372376,
  8: 0.49651918316831684,
  9: 0.4953172937187147,
  10: 0.516927823333604,
  11: 0.4990128331688055,
  12: 0.504852033041944,
  13: 0.4968835701819995,
  14: 0.5011551155115511,
  15: 0.5007905138339921,
  16: 0.5012973137973138,
  17: 0.504878426513861,
  18: 0.5098808266706893,
  19: 0.4929389029284971,
  20: 0.5146420064452851,
  21: 0.4962745232073279,
  22: 0.5070559610705596},
 'Financials': {0: 0.498618057383522,
  1: 0.4904595373949322,
  2: 0.5088226615520802,
  3: 0.4895719620798621,
  4: 0.4959862385321101,
  5: 0.48956636005256243,
  6: 0.5096995444541127,
  7: 0.5089085725921401,
  8: 0.4992282798108041,
  9: 0.5079484479213681,
  10: 0.5024968789013733,
  11: 0.501265965228155,
  12: 0.501370026209197,
  13: 0.5041495014627739,
  14: 0.4840451055662188,
  15: 0.5

['Healthcare', 'Financials', 'Industrials', 'Consumer Cyclicals', 'Technology', 'Consumer Non-Cyclicals', 'Energy', 'Utilities', 'Real Estate', 'Basic Materials']


In [ ]:
dax_sector = pd.read_csv('/raw_data/dax_industry_all.csv')
dax_sector_list = dax_sector["TRBC Economic Sector Name"].unique()
dax_sector_list = [x for x in dax_sector_list if pd.notnull(x)]
print(dax_sector_list)

timesteps = 240
num_input =1
num_classes=1
label = list(range(timesteps)) + ['target'] + ['ticker'] + ['target_date'] + ['sector']

training_data = []
training_label = []
testing_data =[]
testing_label =[]

accuracy_results = {}


for sector in dax_sector_list:
    accuracy_results[sector] = {}  # Create a sub-dictionary for each sector
    for i in range(0,23):
        # read the data
        data_path = '/dax_data/'
        train = pd.read_csv(data_path + 'Set_' + str(i) + '_Train.csv', index_col=0).dropna()
        train = train[train['sector']==sector]
        test = pd.read_csv(data_path + 'Set_' + str(i) + '_Test.csv', index_col=0).dropna()
        test = test[test['sector']==sector]
        
        train.columns = label
        test.columns = label

        train_label = train.iloc[:, timesteps]
        train_data = train.iloc[:, :timesteps]
        test_label = test.iloc[:,timesteps]
        test_data = test.iloc[:, :timesteps]


         # reshape input
        #  data: (samples, timesteps, features)
        x_train = np.array(train_data).reshape((len(train_data), timesteps, num_input), order = 'F')
        x_test = np.array(test_data).reshape((len(test_data), timesteps, num_input), order = 'F')
        # label: (samples, target)
        y_train = np.array(train_label).reshape((len(train_label), num_classes))
        y_test = np.array(test_label).reshape((len(test_label), num_classes))

        print(x_train.shape)
        print(y_train.shape)
        print(x_test.shape)
        print(y_test.shape)

        print("-------------------------------------------------------------------------------------------------------")
        print("Training the model for Training Set " + str(i) + " from " +
        datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'))
        print("-------------------------------------------------------------------------------------------------------")

        if i==0:    
            tuner = keras_tuner.BayesianOptimization(MyHyperModel(),
                    objective='val_accuracy',  overwrite=True,
                    max_trials=30, directory='fnn'+sector, seed=11)
            early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10)
            tuner.search(x_train,y_train, epochs =100,validation_split=0.2,callbacks=[early_stop])

            # save the best model
            hypermodel = MyHyperModel()
            best_hp = tuner.get_best_hyperparameters()[0]
            print(tuner.get_best_hyperparameters()[0].get_config()["values"])
            best_model = hypermodel.build(best_hp)
            early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10)
            result = best_model.fit(x_train,y_train, epochs=1000, validation_split =0.2, verbose =1, callbacks=[early_stop])


        else:
            load_path="/DAX/4_FNN/fnn_model_weight/fnn_weight"+sector+"".h5"
            print('Model restore from '+load_path)
            cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=load_path,
                                                         save_weights_only=True,
                                                         verbose=1)
            early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                            patience = 10)
            result = best_model.fit(
                    x_train, 
                    y_train, 
                    epochs = 1000, 
                    validation_split=0.2,
                    verbose =1,
                    callbacks=[cp_callback, early_stop]        
                )         



        save_path = '/DAX/4_FNN/fnn_model_weight/fnn_weight_'+sector+'.h5'
        best_model.save_weights(save_path)
        print("Model saved to " + save_path)
        print("Training end: " + datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'))

        ##make prediction
        pred_ff_test = best_model.predict(x_test)
        pred = pred_ff_test.reshape((1, len(pred_ff_test))).tolist()[0]
        output_data = pd.DataFrame({'y_prob': pred, 'y_true': test['target'], 'Ticker': test['ticker'],
                                        'Date': test['target_date'], 'Sector': test['sector'], })
        accuracy = accuracy_score(np.round(output_data['y_prob']), output_data['y_true'])
        print('Overall Accuracy for test set:'+ str(accuracy))
        accuracy_results[sector][i] = accuracy

        output_data.to_csv('/DAX/4_FNN/fnn_prediction/fnn_'+sector + '_prediction_period_'+str(i)+'.csv')
        print('Prediction for period ' +sector + str(i) + ' successfully saved.')
        
accuracy_df = pd.DataFrame(accuracy_results)

In [10]:
combined_data_sector = pd.DataFrame()

# Loop through the files and append their conten
for sector in dax_sector_list:
    for i in range(5):  
        path = 'fnn_'+sector + '_prediction_period_'+str(i)+'.csv'
        data = pd.read_csv(path)  
        combined_data_sector = pd.concat([combined_data_sector,data], ignore_index=True)

# Save the combined data to a single CSV file
combined_data_sector.to_csv('dax_fnn_sector_prediction.csv', index=False)